In [1]:
from fastai.vision.all import *
import pandas as pd


In [2]:
def get_landmark(path):
    return path.split(".")[0]

In [3]:
all_image_files = get_image_files("./eight-categories/")

In [4]:
len(all_image_files)

480

In [5]:
dls = ImageDataLoaders.from_name_func(
    ".", all_image_files, valid_pct=0.2, seed=42, label_func=get_landmark, item_tfms=Resize(512), bs=8
)

In [6]:
import json
with open("eight-categories-vocab.json", "w") as f: f.write(json.dumps(list(dls.vocab)))
list(dls.vocab)

['bay-bridge',
 'castro',
 'coit-tower',
 'golden-gate-bridge',
 'port-of-oakland',
 'scribd-logo',
 'sutro-tower',
 'transamerica-pyramid']

In [7]:
from torchvision.models import efficientnet_v2_l, efficientnet_v2_m, efficientnet_v2_s, mobilenet_v3_large
learn_sq = vision_learner(dls, squeezenet1_1, metrics=[accuracy, top_k_accuracy])
learn_sq.fine_tune(5)

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.603406,0.240638,0.906250,1.000000,00:30


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,0.519399,0.045561,0.989583,1.000000,00:35
1,0.398297,0.043436,0.989583,1.000000,00:36
2,0.299332,0.021851,0.989583,1.000000,00:39
3,0.209911,0.003375,1.000000,1.000000,00:38
4,0.160903,0.003537,1.000000,1.000000,00:39


In [8]:
learn_sq.save("eight-categories-sq-512")

Path('models/eight-categories-sq-512.pth')

In [9]:
pytorch_model = learn_sq.model.eval()

In [10]:
import torchvision.transforms
softmax_layer = torch.nn.Softmax(dim=1)
normalization_layer = torchvision.transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
final_model = nn.Sequential(normalization_layer, pytorch_model, softmax_layer)
torch.onnx.export(
    final_model, 
    torch.randn(1, 3, 512, 512).cuda(),
    'landmarks-eight-categories.onnx',
    do_constant_folding=True,
    export_params=True,
    input_names=['input'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                 'output' : {0 : 'batch_size'}}
)
